# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 6 Assignment: Image Processing**

**Student Name: Julia Huang**

# Assignment Instructions

For this assignment you will use two images: 

* [Dog House](https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/hickory_home.jpg)
* [Land Scape](https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/landscape.jpg)


Your code should work with any image; however, these are the two that the **submit** function is expecting.  The goal is to convert both images into square-sized.  In this module we saw how to convert to a square by cropping.  This time we will convert to a square by adding space.  If an image is [landscape orientation](https://en.wikipedia.org/wiki/Page_orientation) you will need to add space at the top and bottom.  Similarly for portrait (taller than wide) you will add space at the sides.  Make sure that the image is centered between the space. 

The following diagram illustrates this.

![Image Processing Instructions](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/image-instructions.png "Image Processing Instructions")

To calculate the color to add to the space, take the average of all RGB values.  Essentially sum all the red values, green, and blue and divide by total number of pixels.  Notice how the darker landscape picture above has a darker color added to the above/below space?  This is due to this averaging.  Make sure you convert your average RGB to integer, RGB does not have fractional values.

The submit function will check to see if your height and width match my solution.  If your height and width are non-square or do not match my dimensions, you likely have a problem with your assignment.  

The submit function also takes three pixels and tests them.  Pixels 1 and 3 are the upper left and lower-right, these are the average color and should match my solution exactly. You might see a difference in pixel 2, which is in the center, if you center the image differently than I do.  If you want to match my solution, make sure to round to integer after any divisions. 


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [0]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))
        

# These functions are provided to build a submission dataframe from the two images that you must
# generate for this assignment.  It is unlikely that you would need to modify these.
def scan_pixel(d,img_array,img_num,pix_num,x,y):
    d[f'img{img_num}-p{pix_num}-rgb0'] = [img_array[y,x,0]]
    d[f'img{img_num}-p{pix_num}-rgb1'] = [img_array[y,x,1]]
    d[f'img{img_num}-p{pix_num}-rgb2'] = [img_array[y,x,2]]

def scan_image(d,img_num,img):
    img_array = np.asarray(img)
    rows = img_array.shape[0]
    cols = img_array.shape[1]
    d[f'img{img_num}-height'] = [rows]
    d[f'img{img_num}-width'] = [cols]
    scan_pixel(d,img_array,img_num,0,0,0)
    scan_pixel(d,img_array,img_num,1,int(cols/2),int(rows/2))
    scan_pixel(d,img_array,img_num,2,cols-1,rows-1)

def build_submit(submit_img1, submit_img2):
    d = {}
    scan_image(d,1,submit_img1)
    scan_image(d,2,submit_img2)
    return pd.DataFrame(d)

# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process.  Running the following code will map your GDrive to /content/drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls /content/drive/My\ Drive/Colab\ Notebooks

assignment_jhuang_class1.ipynb	assignment_jhuang_class6.ipynb
assignment_jhuang_class2.ipynb	assignment_juliahuang_class3.ipynb
assignment_jhuang_class3.ipynb	Untitled
assignment_jhuang_class4.ipynb	Untitled0.ipynb
assignment_jhuang_class5.ipynb


# Assignment #6 Sample Code

The following code provides a starting point for this assignment.

In [0]:
%matplotlib inline

import os
import pandas as pd
import io
import requests
import numpy as np
from scipy.stats import zscore
from PIL import Image, ImageFile
from matplotlib.pyplot import imshow
import requests
from io import BytesIO
import numpy as np

# This is your student key that I emailed to you at the beginnning of the semester.
key = "Yg3Uc8sn118A6HaWAFSKG5g1Th1nOyw34jLD5Uh8"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/My Drive/Colab Notebooks/assignment_jhuang_class6.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class6.ipynb'  # Windows
#file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class6.ipynb'  # Mac/Linux

def fill_square_image(img):
    cols,rows = img.size
    
    
    img = Image.open(BytesIO(response.content))
    img.load()

    img_array = np.asarray(img)
    
    # Create new image
    is_rows_larger_than_cols = True if rows > cols else False
    img2_array = np.zeros((max(rows, cols), max(rows, cols), 3), dtype=np.uint8)
    padding_size = abs(int((rows-cols)/2))
    

    for row in range(rows):
        for col in range(cols):
            t = np.mean(img_array[row,col])
            if is_rows_larger_than_cols:
              img2_array[row,col+padding_size] = img_array[row,col]
            else:
              img2_array[row+padding_size,col] = img_array[row,col]

    median = (np.mean(img_array[:, :, 0]),np.mean(img_array[:, :, 1]),np.mean(img_array[:, :, 2]))

    
    if not is_rows_larger_than_cols:
      img2_array[-padding_size:,:, 0].fill(median[0])
      img2_array[-padding_size:,:, 1].fill(median[1])
      img2_array[-padding_size:,:, 2].fill(median[2])
      img2_array[:padding_size,:, 0].fill(median[0])
      img2_array[:padding_size,:, 1].fill(median[1])
      img2_array[:padding_size,:, 2].fill(median[2])
    else:
      img2_array[:, -padding_size:, 0].fill(median[0])
      img2_array[:, -padding_size:, 1].fill(median[1])
      img2_array[:, -padding_size:, 2].fill(median[2])
      img2_array[:, :padding_size:, 0].fill(median[0])
      img2_array[:, :padding_size, 1].fill(median[1])
      img2_array[:, :padding_size, 2].fill(median[2])
  
    img = Image.fromarray(img2_array, 'RGB')
    
    return img
  
url = "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/hickory_home.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.load()
submit_img1 = fill_square_image(img)
display(submit_img1)


url = "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/landscape.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.load()
submit_img2 = fill_square_image(img)
display(submit_img2)

submit_df = build_submit(submit_img1, submit_img2)
submit(source_file=file,data=submit_df,key=key,no=6)
    



In [0]:
def fill_square_image(img):
    cols,rows = img.size
    
    
    img = Image.open(BytesIO(response.content))
    img.load()

    img_array = np.asarray(img)
    
    # Create new image
    is_rows_larger_than_cols = True if rows > cols else False
    img2_array = np.zeros((max(rows, cols), max(rows, cols), 3), dtype=np.uint8)
    padding_size = abs(int((rows-cols)/2))
    

    for row in range(rows):
        for col in range(cols):
            t = np.mean(img_array[row,col])
            if is_rows_larger_than_cols:
              img2_array[row,col+padding_size] = img_array[row,col]
            else:
              img2_array[row+padding_size,col] = img_array[row,col]

    median = (np.mean(img_array[:, :, 0]),np.mean(img_array[:, :, 1]),np.mean(img_array[:, :, 2]))

    
    if not is_rows_larger_than_cols:
      img2_array[-padding_size:,:, 0].fill(median[0])
      img2_array[-padding_size:,:, 1].fill(median[1])
      img2_array[-padding_size:,:, 2].fill(median[2])
      img2_array[:padding_size,:, 0].fill(median[0])
      img2_array[:padding_size,:, 1].fill(median[1])
      img2_array[:padding_size,:, 2].fill(median[2])
    else:
      img2_array[:, -padding_size:, 0].fill(median[0])
      img2_array[:, -padding_size:, 1].fill(median[1])
      img2_array[:, -padding_size:, 2].fill(median[2])
      img2_array[:, :padding_size:, 0].fill(median[0])
      img2_array[:, :padding_size, 1].fill(median[1])
      img2_array[:, :padding_size, 2].fill(median[2])
  
    img = Image.fromarray(img2_array, 'RGB')
    
    return img

url = "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/hickory_home.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.load()
submit_img1 = fill_square_image(img)
display(submit_img1)


url = "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/landscape.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.load()
submit_img2 = fill_square_image(img)
display(submit_img2)

# images = ["https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/hickory_home.jpg", "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/landscape.jpg"]
    
# for url in images:
#     ImageFile.LOAD_TRUNCATED_IMAGES = False
#     response = requests.get(url)
#     img = Image.open(BytesIO(response.content))
#     img.load()
#     img = fill_square_image(img)
#     img = img.resize((256,256), Image.ANTIALIAS)
    
#     display(img)
    





In [0]:
import os
import pandas as pd
import io
import requests
import numpy as np
from scipy.stats import zscore
from PIL import Image, ImageFile
from matplotlib.pyplot import imshow
import requests
from io import BytesIO
import numpy as np
import cv2



def fill_square_image(img):
  
  cols,rows = img.size
    
    
  img = Image.open(BytesIO(response.content))
  img.load()

  img_array = np.asarray(img)
    
  # Create new image
  is_rows_larger_than_cols = True if rows > cols else False
  img2_array = np.zeros((max(rows, cols), max(rows, cols), 3), dtype=np.uint8)
  padding_size = abs(int((rows-cols)/2))
    

  for row in range(rows):
    for col in range(cols):
      t = np.mean(img_array[row,col])
      if is_rows_larger_than_cols:
        img2_array[row,col+padding_size] = img_array[row,col]
      else:
        img2_array[row+padding_size,col] = img_array[row,col]

  median = (np.mean(img_array[:, :, 0]),np.mean(img_array[:, :, 1]),np.mean(img_array[:, :, 2]))

    
  if not is_rows_larger_than_cols:
    img2_array[-padding_size:,:, 0].fill(median[0])
    img2_array[-padding_size:,:, 1].fill(median[1])
    img2_array[-padding_size:,:, 2].fill(median[2])
    img2_array[:padding_size,:, 0].fill(median[0])
    img2_array[:padding_size,:, 1].fill(median[1])
    img2_array[:padding_size,:, 2].fill(median[2])
  else:
    img2_array[:, -padding_size:, 0].fill(median[0])
    img2_array[:, -padding_size:, 1].fill(median[1])
    img2_array[:, -padding_size:, 2].fill(median[2])
    img2_array[:, :padding_size:, 0].fill(median[0])
    img2_array[:, :padding_size, 1].fill(median[1])
    img2_array[:, :padding_size, 2].fill(median[2])
  
    img = Image.fromarray(img2_array, 'RGB')
  return img

url = "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/hickory_home.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.load()
img = img.resize((256,256), Image.ANTIALIAS)
submit_img1 = fill_square_image(img)
display(submit_img1)


url = "https://github.com/jeffheaton/t81_558_deep_learning/raw/master/photos/landscape.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.load()
img = img.resize((256,256), Image.ANTIALIAS)
submit_img2 = fill_square_image(img)
display(submit_img2)





IndexError: ignored

In [0]:
# Submit
submit_df = build_submit(submit_img1, submit_img2)
submit(source_file=file,data=submit_df,key=key,no=6)